In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import tensorflow as tf
import numpy as np
from keras import layers
import pandas as pd

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("gcloud_project_name")

In [ ]:
from google.cloud import bigquery
bigquery_client = bigquery.Client(project=secret_value_0)
from google.cloud import storage
storage_client = storage.Client(project=secret_value_0)

In [ ]:
bigquery_client.query(f"SELECT * FROM `{secret_value_0}.detect_llm_ds_bq.raw_data` LIMIT 100").result().to_dataframe()

In [ ]:
job_config = bigquery.QueryJobConfig(destination=f"{secret_value_0}.detect_llm_ds_bq.shuffle_raw",write_disposition="WRITE_TRUNCATE")
sql = f'SELECT * \
FROM `{secret_value_0}.detect_llm_ds_bq.raw_data` ORDER BY RAND()'
query_job = bigquery_client.query(sql, job_config=job_config)  
query_job.result()  
print("Done")

In [ ]:
job_config = bigquery.QueryJobConfig(destination=f"{secret_value_0}.detect_llm_ds_bq.training",write_disposition="WRITE_TRUNCATE")
sql = f'SELECT * \
FROM `{secret_value_0}.detect_llm_ds_bq.shuffle_raw` AS train \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(train))), 12) IN (0, 2, 3, 4 ,5,6,7,8,9,10,11)'
query_job = bigquery_client.query(sql, job_config=job_config)  
query_job.result()  
print("Done")

In [ ]:
job_config = bigquery.QueryJobConfig(destination=f"{secret_value_0}.detect_llm_ds_bq.test",write_disposition="WRITE_TRUNCATE")
sql = f'SELECT * \
FROM `{secret_value_0}.detect_llm_ds_bq.shuffle_raw` AS train \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(train))), 12) IN (1,12)'
query_job = bigquery_client.query(sql, job_config=job_config)  
query_job.result()  
print("Done")

In [ ]:
job_config = bigquery.QueryJobConfig(destination=f"{secret_value_0}.detect_llm_ds_bq.validation",write_disposition="WRITE_TRUNCATE")
sql = f'SELECT * \
FROM `{secret_value_0}.detect_llm_ds_bq.training` AS train \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(train))), 12) IN (11,12)'
query_job = bigquery_client.query(sql, job_config=job_config)  
query_job.result()  
print("Done")

In [ ]:
job_config = bigquery.QueryJobConfig(destination=f"{secret_value_0}.detect_llm_ds_bq.training",write_disposition="WRITE_TRUNCATE")
sql = f'SELECT * \
FROM `{secret_value_0}.detect_llm_ds_bq.training` AS train \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(train))), 12) IN (0,1,2,3,4,5,6,7,8,9,10)'
query_job = bigquery_client.query(sql, job_config=job_config)  
query_job.result()  
print("Done")

In [75]:
train0 = bigquery_client.query(f"SELECT * FROM `{secret_value_0}.detect_llm_ds_bq.training`").result().to_dataframe()

In [76]:
val0 = bigquery_client.query(f"SELECT * FROM `{secret_value_0}.detect_llm_ds_bq.validation`").result().to_dataframe()

In [77]:
test0 = bigquery_client.query(f"SELECT * FROM `{secret_value_0}.detect_llm_ds_bq.test`").result().to_dataframe()

In [78]:
train0.label.value_counts()

label
0    22825
1    14603
Name: count, dtype: int64

In [79]:
val0.label.value_counts()

label
0    2321
1    1482
Name: count, dtype: int64

In [80]:
test0.label.value_counts()

label
0    2225
1    1412
Name: count, dtype: int64

In [81]:
seed = 2 
dataset_tr = tf.data.Dataset.from_tensor_slices((train0.text.values,train0.label.values ))
dataset_tr = dataset_tr.shuffle(buffer_size=len(train0)).batch(batch_size=2)

dataset_val = tf.data.Dataset.from_tensor_slices((val0.text.values,val0.label.values ))
dataset_val = dataset_val.shuffle(buffer_size=len(val0)).batch(batch_size=2)

dataset_test = tf.data.Dataset.from_tensor_slices((test0.text.values))
dataset_test = dataset_test.batch(batch_size=2)


In [82]:
import string
import re
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )
max_features = 20000
embedding_dim = 128
sequence_length = 500
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_ds = dataset_tr.map(lambda x, y: x).concatenate(dataset_val.map(lambda x, y: x)).concatenate(dataset_test)
vectorize_layer.adapt(text_ds)

Cause: could not parse the source code of <function <lambda> at 0x7e0895040550>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x, y: x

Match 1:
lambda x, y: x

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7e08c0912b00>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
lambda x, y: x

Match 1:
lambda x, y: x

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [83]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label
def vectorize_text_test(text):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text)
train_ds = dataset_tr.map(vectorize_text)
train_ds = train_ds.cache().prefetch(buffer_size=10)

val_ds = dataset_val.map(vectorize_text)
val_ds = val_ds.cache().prefetch(buffer_size=10)

test_ds = dataset_test.map(vectorize_text_test)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [84]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)
model = keras.Model(inputs, predictions)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy",tf.keras.metrics.AUC()])

In [86]:
num_cpus = os.cpu_count()

In [85]:
epochs = 1
tf.random.set_seed(1)
model.fit(train_ds, validation_data=val_ds, epochs=epochs,workers=num_cpus)

18714/18714 [==============================] - 496s 26ms/step - loss: 0.0816 - accuracy: 0.9728 - auc: 0.9936 - val_loss: 0.0248 - val_accuracy: 0.9921 - val_auc: 0.9990


In [87]:
model.evaluate(val_ds)

1902/1902 [==============================] - 8s 4ms/step - loss: 0.0248 - accuracy: 0.9921 - auc: 0.9990


[0.02477864734828472, 0.9921115040779114, 0.9989569783210754]

In [88]:
preds = model.predict(test_ds)

1819/1819 [==============================] - 7s 4ms/step


In [89]:
from sklearn.metrics import roc_auc_score

In [91]:
roc_auc_score(test0.label,preds)

0.9991857911321895

In [99]:
test.text[123]

"If you are someone who likes to go exploring and work a little then here is the job for you. The reason you should join this program is because on your extra time you can go sight seeing and look at what you will be passing on the jorney from place to place. The task you are asked to do is simple and you can do a variety of things on your way back home.\n\nIf you join this program then you could travel half way around the world, see different things, be in the millitary, help people, and work all at the same time. When you join this program you can see things like ruins and famos statues. Luke joined and look what he seen. He seen Europe, China, an excavated castle, and the Panama Canal. You don't see those things on an average day. If you join you can see all these things and have a life time expereince.\n\nWhen you are on this voage all you have to do is feed the animals and water them two or three times a day. The bales of hay and bags of oats had to be pulled up from the lower hol

In [100]:
inputs = keras.Input(shape=(1,), dtype="string")
indices = vectorize_layer(inputs)
outputs = model(indices)
end_to_end_model = keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy",tf.keras.metrics.AUC()]
)
end_to_end_preds = end_to_end_model.predict([test.text[123]])

1/1 [==============================] - 0s 180ms/step


In [104]:
test.label[123],end_to_end_preds.round(3)

(0, array([[0.]], dtype=float32))

In [126]:
end_to_end_model.save("end_to_end_detect_llm.keras",save_format="keras")

```python
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )
```

In [128]:
loaded_model = keras.models.load_model("end_to_end_detect_llm.keras"
                                      ,custom_objects={"custom_standardization":custom_standardization})

In [129]:
end_to_end_model.predict(["If you are someone who likes to go exploring and work a little then here is the job for you. The reason you should join this program is because on your extra time you can go sight seeing and look at what you will be passing on the jorney from place to place. The task you are asked to do is simple and you can do a variety of things on your way back home.\n\nIf you join this program then you could travel half way around the world, see different things, be in the millitary, help people, and work all at the same time. When you join this program you can see things like ruins and famos statues. Luke joined and look what he seen. He seen Europe, China, an excavated castle, and the Panama Canal. You don't see those things on an average day. If you join you can see all these things and have a life time expereince.\n\nWhen you are on this voage all you have to do is feed the animals and water them two or three times a day. The bales of hay and bags of oats had to be pulled up from the lower holds of the ship, so you might need some mucles. You also have to clean the stalls daily, which I know is a nasty job but someone has to do it. You also get the benefit of hellping families in need.\n\nOn the way back home you can also have fun by playing games with your friends in the empty stalls. It might not be the most sanatairy place but what else is there to do. Luke says that he and his friends play: Table-tennis, fencing, boxing,voley ball, base ball, and reading. This helped pass the time of the long jorney back home. This program could be fun even if you are in a ship and in animal stalls.\n\nThis why I think that joining this program would be good. You get to go sighting seeing, work, and have fun at the same time. Who would want to miss this opportunity of a life time like this one.         "])

1/1 [==============================] - 0s 50ms/step


array([[2.3142486e-06]], dtype=float32)

In [130]:
loaded_model.predict(["If you are someone who likes to go exploring and work a little then here is the job for you. The reason you should join this program is because on your extra time you can go sight seeing and look at what you will be passing on the jorney from place to place. The task you are asked to do is simple and you can do a variety of things on your way back home.\n\nIf you join this program then you could travel half way around the world, see different things, be in the millitary, help people, and work all at the same time. When you join this program you can see things like ruins and famos statues. Luke joined and look what he seen. He seen Europe, China, an excavated castle, and the Panama Canal. You don't see those things on an average day. If you join you can see all these things and have a life time expereince.\n\nWhen you are on this voage all you have to do is feed the animals and water them two or three times a day. The bales of hay and bags of oats had to be pulled up from the lower holds of the ship, so you might need some mucles. You also have to clean the stalls daily, which I know is a nasty job but someone has to do it. You also get the benefit of hellping families in need.\n\nOn the way back home you can also have fun by playing games with your friends in the empty stalls. It might not be the most sanatairy place but what else is there to do. Luke says that he and his friends play: Table-tennis, fencing, boxing,voley ball, base ball, and reading. This helped pass the time of the long jorney back home. This program could be fun even if you are in a ship and in animal stalls.\n\nThis why I think that joining this program would be good. You get to go sighting seeing, work, and have fun at the same time. Who would want to miss this opportunity of a life time like this one.         "])

1/1 [==============================] - 0s 201ms/step


array([[2.3142486e-06]], dtype=float32)